models_plan = """
For TSLA Stock Price Forecasting:
1. ARIMA/SARIMA - Good for capturing trends and seasonality
2. Prophet (Facebook) - Handles missing data and holidays well
3. LSTM (Deep Learning) - Captures complex non-linear patterns
4. Ensemble - Combine multiple models for robustness

For Wealth Projection:
1. Monte Carlo Simulation - Account for uncertainty
2. Scenario Analysis - Bull/Base/Bear cases
3. Regression Models - Model wealth as f(TSLA_price, Other_growth)

Trillionaire Timeline Calculation:
- Project both components separately
- Add them together for total wealth projection
- Calculate when trajectory crosses $1 trillion
- Provide confidence intervals

In [ ]:

Step 2: Generate 5-year forecast (60 months)
Step 3: Calculate TSLA price projections
Step 4: Project Elon's TSLA wealth component
Step 5: Project Other Wealth component (SpaceX, xAI)
Step 6: Calculate total wealth trajectory
Step 7: Determine when $1 TRILLION is reached 🎯
Step 8: Create Bull/Base/Bear scenarios
Step 9: Monte Carlo for confidence intervals

In [ ]:
# Calculate Elon's wealth trajectory

print("\n" + "=" * 70)
print("STEP 8: ELON MUSK WEALTH PROJECTION")
print("=" * 70)

# Current parameters (as of Nov 2025)
current_ownership = merged_data['Ownership_Percentage'].iloc[-1] / 100  # Convert to decimal
current_shares = merged_data['Musk_Shares_Millions'].iloc[-1]  # Millions
current_other_wealth = merged_data['Other_Wealth_Billions'].iloc[-1]  # Billions

# Calculate Other Wealth growth rate (historical average)
other_wealth_growth_rate = merged_data['Other_Wealth_Growth'].mean()

print(f"\nCurrent Parameters:")
print(f"  TSLA Ownership: {current_ownership*100:.2f}%")
print(f"  Shares Owned: {current_shares:.1f} million")
print(f"  Other Wealth (SpaceX, xAI, etc.): ${current_other_wealth:.1f}B")
print(f"  Avg Other Wealth Growth Rate: {other_wealth_growth_rate:.4f} per month")

# Project future wealth
wealth_projection = []

for i, date in enumerate(forecast_df.index):
    # TSLA wealth component
    tsla_price = forecast_df['Forecast_Price'].iloc[i]
    tsla_wealth = (tsla_price * current_shares) / 1000  # Convert to billions
    
    # Other wealth component (grows at historical rate)
    months_forward = i + 1
    other_wealth = current_other_wealth * ((1 + other_wealth_growth_rate) ** months_forward)
    
    # Total wealth
    total_wealth = tsla_wealth + other_wealth
    
    wealth_projection.append({
        'Date': date,
        'TSLA_Price': tsla_price,
        'TSLA_Wealth_B': tsla_wealth,
        'Other_Wealth_B': other_wealth,
        'Total_Wealth_B': total_wealth
    })

wealth_df = pd.DataFrame(wealth_projection)
wealth_df.set_index('Date', inplace=True)

print("\nWealth Projection Summary:")
print(wealth_df.head(12))
print("...")
print(wealth_df.tail(12))

In [ ]:
# Assessing Elon's Trillionaire Timeline

print("\n" + "=" * 70)
print("🎯 WHEN WILL ELON MUSK BECOME A TRILLIONAIRE?")
print("=" * 70)

# Find when he crosses $1 trillion
trillion = 1000  # Billions

trillionaire_rows = wealth_df[wealth_df['Total_Wealth_B'] >= trillion]

if len(trillionaire_rows) > 0:
    trillionaire_date = trillionaire_rows.index[0]
    trillionaire_wealth = trillionaire_rows['Total_Wealth_B'].iloc[0]
    tsla_price_at_trillion = trillionaire_rows['TSLA_Price'].iloc[0]
    
    # Calculate time from now
    months_until = (trillionaire_date.year - merged_data.index[-1].year) * 12 + \
                   (trillionaire_date.month - merged_data.index[-1].month)
    years_until = months_until / 12
    
    print(f"\n🚀 PROJECTION RESULTS:")
    print(f"   Trillionaire Date: {trillionaire_date.strftime('%B %Y')}")
    print(f"   Time from Now: {years_until:.1f} years ({months_until} months)")
    print(f"   Projected Wealth: ${trillionaire_wealth:.1f} Billion")
    print(f"   Required TSLA Price: ${tsla_price_at_trillion:.2f}")
    print(f"\n   Breakdown at $1T:")
    print(f"      TSLA Wealth: ${trillionaire_rows['TSLA_Wealth_B'].iloc[0]:.1f}B")
    print(f"      Other Wealth: ${trillionaire_rows['Other_Wealth_B'].iloc[0]:.1f}B")
else:
    print("\n⚠️ Based on current projections, $1 trillion may not be reached within 5 years.")
    print(f"   Max projected wealth: ${wealth_df['Total_Wealth_B'].max():.1f}B")
    print(f"   At: {wealth_df['Total_Wealth_B'].idxmax().strftime('%B %Y')}")
    
    # Calculate what TSLA price would be needed
    current_other = wealth_df['Other_Wealth_B'].iloc[-1]
    needed_tsla_wealth = trillion - current_other
    needed_tsla_price = (needed_tsla_wealth * 1000) / current_shares
    
    print(f"\n   To reach $1T by {forecast_df.index[-1].strftime('%Y')}:")
    print(f"      Required TSLA Price: ${needed_tsla_price:.2f}")
    print(f"      Current Forecast: ${forecast_df['Forecast_Price'].iloc[-1]:.2f}")

# Current status
current_networth = merged_data['Net_Worth_Billions'].iloc[-1]
remaining = trillion - current_networth

print(f"\n📊 CURRENT STATUS:")
print(f"   Current Net Worth: ${current_networth:.1f}B")
print(f"   To $1 Trillion: ${remaining:.1f}B more needed")
print(f"   Progress: {(current_networth/trillion)*100:.1f}%")